# 第5章: 係り受け解析
夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](https://nlp100.github.io/data/neko.txt)）を[CaboCha](https://taku910.github.io/cabocha/)を使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [2]:
!mkdir data
!curl -O https://nlp100.github.io/data/neko.txt
!mv neko.txt ./data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  943k  100  943k    0     0   635k      0  0:00:01  0:00:01 --:--:--  635k


In [8]:
!cat ./data/neko.txt | cabocha -f1 > ./data/neko.txt.cabocha

In [10]:
!head -30 ./data/neko.txt.cabocha

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 -1D 1/1 0.000000
　	記号,空白,*,*,*,*,　,　,　
吾輩は猫である	名詞,固有名詞,一般,*,*,*,吾輩は猫である,ワガハイハネコデアル,ワガハイワネコデアル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
* 2 -1D 0/0 0.000000
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。
EOS
EOS
* 0 1D 1/2 1.504358
　	記号,空白,*,*,*,*,　,　,　
どこ	名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ
で	助詞,格助詞,一般,*,*,*,で,デ,デ
* 1 2D 0/1 1.076607
生れ	動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
* 2 4D 0/1 -0.197109
かとん	名詞,一般,*,*,*,*,火遁,カトン,カトン
と	助詞,格助詞,一般,*,*,*,と,ト,ト
* 3 4D 0/1 -0.197109


## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．
このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．
さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [38]:
IGNORE_CHARACTERS = {'\u3000'}
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1= pos1
    
    def __repr__(self):
        display_surface = self.surface
        return f"Morph({display_surface})"

In [40]:
filepath = 'data/neko.txt.cabocha'
with open(filepath, 'r') as f:
    result_list = []
    line_list = []
    for line in f:
        if line.startswith('*'):
            continue
        if 'EOS' == line.strip():
            if line_list:
                result_list.append(line_list)
            line_list = []
        else:
            surface, others = line.split('\t')
            split_results = others.strip().split(',')
            if 9 == len(split_results):
                pos, pos1, _, _, _, _, base, _, _ = split_results
            elif 7 == len(split_results):
                pos, pos1, _, _, _, _, _ = split_results
                base = None
            
            if surface not in IGNORE_CHARACTERS:
                morph = Morph(surface, base, pos, pos1)
                line_list.append(morph)

display_line_num = 3  # 3行目を表示
print(result_list[display_line_num])

[Morph(どこ), Morph(で), Morph(生れ), Morph(た), Morph(かとん), Morph(と), Morph(見当), Morph(が), Morph(つか), Morph(ぬ), Morph(。)]


## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．
このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．
さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．
第5章の残りの問題では，ここで作ったプログラムを活用せよ

In [78]:
class Chunk(object):
    def __init__(self, chunk_num, dst):
        self.chunk_num = chunk_num
        self.morphs = []
        self.srcs = []
        self.dst = int(dst.replace('D', ''))
    
    def __repr__(self):
        morph_str = ','.join([str(_m) for _m in self.morphs])
        return f"{self.chunk_num}: Chunk({morph_str}) -> {self.dst}"
    
def set_srcs(chunk_list):
    if 1 == len(chunk_list):
        chunk_list[0].srcs.append(-1)
        return chunk_list
    dst_dict = {_i: _c.dst for _i, _c in enumerate(chunk_list) if _c.dst != -1}
    
    for src, dst in dst_dict.items():
        chunk_list[dst].srcs.append(src)
    return chunk_list

filepath = 'data/neko.txt.cabocha'
chunk = None
with open(filepath, 'r') as f:
    result_list = []
    line_list = []
    for line in f:
        if line.startswith('*'):
            if chunk and chunk.morphs:
                line_list.append(chunk)
            _, chunk_num, dst, _, _ = line.strip().split()
            chunk = Chunk(chunk_num, dst)
            continue


        if 'EOS' == line.strip():
            if chunk and chunk.morphs:
                line_list.append(chunk)

            if line_list:
                result_list.append(set_srcs(line_list))
            chunk = None
            line_list = []
        else:
            surface, others = line.split('\t')
            split_results = others.strip().split(',')
            if 9 == len(split_results):
                pos, pos1, _, _, _, _, base, _, _ = split_results
            elif 7 == len(split_results):
                pos, pos1, _, _, _, _, _ = split_results
                base = None
            
#             if surface not in IGNORE_CHARACTERS:
            morph = Morph(surface, base, pos, pos1)
            chunk.morphs.append(morph)

display_line_num = 8  # 8行目を表示
print(result_list[display_line_num-1])

[0: Chunk(Morph(この)) -> 1, 1: Chunk(Morph(書生),Morph(という),Morph(の),Morph(は)) -> 7, 2: Chunk(Morph(時々)) -> 4, 3: Chunk(Morph(我々),Morph(を)) -> 4, 4: Chunk(Morph(捕え),Morph(て)) -> 5, 5: Chunk(Morph(煮),Morph(て)) -> 6, 6: Chunk(Morph(食う),Morph(という)) -> 7, 7: Chunk(Morph(話),Morph(で),Morph(ある),Morph(。)) -> -1]


## 42. 係り元と係り先の文節の表示

## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

## 44. 係り受け木の可視化

## 45. 動詞の格パターンの抽出

## 46. 動詞の格フレーム情報の抽出

## 47. 機能動詞構文のマイニング

## 48. 名詞から根へのパスの抽出

## 49. 名詞間の係り受けパスの抽出